In [138]:
%matplotlib inline
import xarray as xr
import numpy as np
import datetime as dt
from datetime import timedelta
from opendrift.readers import reader_netCDF_CF_generic
from opendrift.readers import reader_global_landmask
from opendrift.models.oceandrift import OceanDrift
import os
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib import cm

# matplotlib.use('TkAgg')
from IPython.display import HTML, display
from mpl_toolkits.axes_grid1 import make_axes_locatable


In [2]:
def make_map():
    fig = plt.figure(figsize=(12,9))
    cart_proj = ccrs.PlateCarree()
    ax = plt.axes(projection=cart_proj)
    ax.coastlines('10m', linewidth=0.8,zorder=200)
    ax.set_xlim(-123.3,-122.2)
    ax.set_ylim(37.5,38.1)
    return(fig, ax)

## Plotting the data ##

__1. Old model runs are found under Data/model_outputs/sf_bay_start_time__
First, lets start by plotting a model run

In [162]:
ds = xr.open_dataset('Data/model_output/sf_bay_continuous_20201204T130000.nc')
df = ds['status'].to_dataframe()
df['start_time'] = 0
df = df.query("trajectory <= 300")

In [163]:
time_steps = df.index.get_level_values(1).values
t = time_steps[48]
dt = time_steps[24]
step = df.query("time <= @t & time >= @dt & status == 0")

In [ ]:
time_steps.unique().shape[0]

In [ ]:
time_steps = df.index.get_level_values(1).values
norm = mcolors.Normalize(vmin=0, vmax=25)
for i in range(time_steps.shape[0]):
    fig, ax = make_map()
    t = time_steps[0]
    if i < 24:
        dt = time_steps[0]
    else:
        dt = time_steps[i -  24]
    step = df.query("time <= @t & time >= @dt & status == 0")
    for traj, temp_df in step.groupby(level=0):
        ax.plot(temp_df['lon'], temp_df['lat'],color='.75')
        mod = traj//100
        marker = temp_df.iloc[-1]
        ax.scatter(marker['lon'],marker['lat'],c=norm(mod),zorder=10)
        plt.savefig(f"Data/model_output/animation_cont/sfBay_cont_01_frame_{i:04}.png")
        plt.close()

In [11]:
max_val = surface_map['density_surface'].max()
fig, ax = make_map()
current_step = surface_map.isel(time=2)
im = current_step['density_surface'].plot(ax=ax, vmin=0, vmax=max_val, add_colorbar=False)
plt.colorbar(im, fraction=0.026, pad=0.02,)
plt.show()

In [12]:
max_val = surface_map['density_surface'].max()

for i in range(surface_map['time'].shape[0]):
    fig, ax = make_map()
    current_step = surface_map.isel(time=i)
    im = current_step['density_surface'].plot(ax=ax, vmin=0, vmax=max_val, add_colorbar=False)
    plt.colorbar(im, fraction=0.026, pad=0.02)
    plt.text(.02,.95,"time_step: {}".format(i), transform=ax.transAxes)
    plt.savefig(f"Data/model_output/animation/sfBay_density_large_01_frame_{i:04}.png")
    plt.close()

In [13]:
!magick convert Data/model_output/animation/sfBay_density_large_01_frame*png Data/model_output/density_animation_large.gif
display(HTML("<img src='Data/model_output/density_animation_large.gif' />"))

/bin/bash: magick: command not found


In [3]:
import glob
from PIL import Image

In [5]:
# filepaths
fp_in = "Data/model_output/animation/sfBay_density_large_01_frame*png"
fp_out = "Data/model_output/animation/density_animation_large.gif"

# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
img, *imgs = [Image.open(f) for f in sorted(glob.glob(fp_in))]
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0)


NameError: name 'o' is not defined